In [103]:
import spacy
import en_core_web_sm
nlp = en_core_web_sm.load(disable=[ "ner","tagger",'textcat'])
from spellchecker import SpellChecker
import nltk
from nltk.tokenize import word_tokenize
import re
import string
import unicodedata
import emoji as emoji
import regex
import spacy
from spacy.tokens import Token
from bs4 import BeautifulSoup
import pandas as pd
from nltk.corpus import stopwords
from tqdm import tqdm
tqdm.pandas()
from sklearn.model_selection import train_test_split
from sklearn import feature_extraction, linear_model, model_selection, preprocessing
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from pandas import Panel
from sklearn.svm import LinearSVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import ParameterGrid
from sklearn.model_selection import KFold
import time
import numpy as np
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from tqdm import tqdm_notebook
import warnings
warnings.filterwarnings("ignore")

C:\Users\asus\Anaconda3\lib\site-packages\ipykernel_launcher.py:30: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version


In [106]:
def DT_grid_search(hyper_comb,X_train,y_train,file_name,c_w):
    start=time.time()
    all_res_cv = []
    cv = KFold(n_splits=5, shuffle=True, random_state=1848773) # 5 fold
    grid = ParameterGrid(hyper_comb) #all possible combination of parameters chosen
    for hypers in tqdm(grid):
        class_weight=hypers['class_weight'],
        criterion=hypers['criterion']
        max_depth=hypers['max_depth']
        max_features=hypers['max_features']
        max_leaf_nodes=hypers['max_leaf_nodes']
        min_impurity_decrease=hypers['min_impurity_decrease']
        min_impurity_split=hypers['min_impurity_split']
        min_samples_leaf=hypers['min_samples_leaf']
        min_samples_split=hypers['min_samples_split']
        min_weight_fraction_leaf=hypers['min_weight_fraction_leaf']
        presort=hypers['presort']
        splitter=hypers['splitter']
        val_f1_list = []
        train_f1_list = []
        val_recall_list=[]
        train_recall_list=[]
        val_accuracy_list=[]
        train_accuracy_list=[]
        val_pre_list=[]
        train_pre_list=[]
        

        for train_index, test_index in (cv.split(X_train)): # cross validation
            #start1=time.time()
            #take 80% and 20%
            X_train_cv, X_val, y_train_cv, y_val = X_train[train_index], X_train[test_index], y_train[train_index], y_train[test_index]
            #fit the model with the chosen param1848773eters
            model=tree.DecisionTreeClassifier(class_weight=c_w, criterion=criterion, max_depth=max_depth,
                       max_features=max_features, max_leaf_nodes=max_leaf_nodes,
                       min_impurity_decrease=min_impurity_decrease, min_impurity_split=min_impurity_split,
                       min_samples_leaf=min_samples_leaf, min_samples_split=min_samples_split,
                       min_weight_fraction_leaf=min_weight_fraction_leaf, presort=presort,
                       random_state=1848773, splitter=splitter).fit(X_train_cv,y_train_cv)

            # make prediction and compute F1
            pred_train = model.predict(X_train_cv)
            pred_val = model.predict(X_val)
            #F1 SCORE
            f1_train = f1_score(y_train_cv, pred_train, average='micro')
            train_f1_list.append(f1_train)
            f1_val = f1_score(y_val, pred_val, average='micro')
            val_f1_list.append(f1_val)
            #RECALL VALUES
            recall_train=recall_score(y_train_cv,pred_train, average='micro')
            train_recall_list.append(recall_train)
            recall_val=recall_score(y_val,pred_val, average='micro')
            val_recall_list.append(recall_val)
            #ACCURACY
            accuracy_train=accuracy_score(y_train_cv,pred_train)
            train_accuracy_list.append(accuracy_train)
            accuracy_val=accuracy_score(y_val,pred_val)
            val_accuracy_list.append(accuracy_val)
            #PRECISION
            pre_train=precision_score(y_train_cv,pred_train, average='micro')
            train_pre_list.append(pre_train)
            pre_val=precision_score(y_val,pred_val, average='micro')
            val_pre_list.append(pre_val)
            #end1=time.time()

        #print(end1-start1)
        all_res_cv.append({ 'class_weight':class_weight,
            'criterion':criterion,
            'max_depth':max_depth,
            'max_features':max_features,
            'max_leaf_nodes':max_leaf_nodes,
            'min_impurity_decrease':min_impurity_decrease,
            'min_impurity_split':min_impurity_split,
            'min_samples_leaf':min_samples_leaf,
            'min_samples_split':min_samples_split,
            'min_weight_fraction_leaf':min_weight_fraction_leaf,
            'presort':presort,
            'splitter':splitter,
        'val_f1':np.mean(val_f1_list), 
        'train_f1':np.mean(train_f1_list),
        'val_recall':np.mean(val_recall_list), 
        'train_recall':np.mean(train_recall_list),
        'val_acc':np.mean(val_accuracy_list), 
        'train_acc':np.mean(train_accuracy_list),
        'val_pre':np.mean(val_pre_list),
        'train_pre':np.mean(train_pre_list)})
        '''
        print(({'class_weight':class_weight,
            'criterion':criterion,
            'max_depth':max_depth,
            'max_features':max_features,
            'max_leaf_nodes':max_leaf_nodes,
            'min_impurity_decrease':min_impurity_decrease,
            'min_impurity_split':min_impurity_split,
            'min_samples_leaf':min_samples_leaf,
            'min_samples_split':min_samples_split,
            'min_weight_fraction_leaf':min_weight_fraction_leaf,
            'presort':presort,
            'splitter':splitter,
            'val_f1':np.mean(val_f1_list), 'train_f1':np.mean(train_f1_list)}))
            '''


        # after the 5th iteration we make the mean of the collected errors to find a more reliable value for the error
    end=time.time()-start
    df = pd.DataFrame(all_res_cv) # collect all results in a dataframe
    df.to_csv(file_name, index=False) # save dataframe in csv with results

    return print('File created grid_search_cv')

##combination of hyperparameters

In [55]:
train=pd.read_csv('pre_train.csv')

In [56]:
sub=list(train['prep_text'])
sub[5115]='Err:509'
train['prep_text']=sub

In [102]:
for i in tqdm([8,2,2,3]):
    print(i)

  0%|          | 0/4 [00:00<?, ?it/s]

8
2
2
3


100%|██████████| 4/4 [00:00<00:00, 501.35it/s]


In [99]:
!pip install tqdm

## bow_models

In [57]:
X = train['prep_text']
y = train['target']

In [58]:
count_vectorizer = feature_extraction.text.CountVectorizer()
X_train_=count_vectorizer.fit_transform(X)

In [77]:
hyper_comb={'class_weight':[None],
            'criterion':['gini','entropy'],
            'max_depth':[None],
            'max_features':['auto','sqrt','log2',None],
            'max_leaf_nodes':[None],
            'min_impurity_decrease':[0],
            'min_impurity_split':[1e-6,1e-7],
            'min_samples_leaf':list(np.arange(0.1,0.5,0.15))+[2,3,4,5],
            'min_samples_split':list(np.arange(0.1,1,0.15))+[2,3,4,5],
            'min_weight_fraction_leaf':[0.0],
            'presort':[False],
            'splitter':['best','random']}

#class_weight=None
DT_grid_search(hyper_comb,X_train_,y,'DT_grid_search_cv_bow_none.csv',None)

In [84]:
max(list(pd.read_csv('DT_grid_search_cv_bow_none.csv')['val_f1']))

0.7697355399425195

In [107]:
#class_weight=balanced
DT_grid_search(hyper_comb,X_train_,y,'DT_grid_search_cv_bow_balanced.csv','balanced')

100%|██████████| 2240/2240 [27:58<00:00,  1.33it/s]  


File created grid_search_cv


In [108]:
max(list(pd.read_csv('DT_grid_search_cv_bow_balanced.csv')['val_f1']))

0.7639563486893477

In [15]:
tree.DecisionTreeClassifier()

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

### tf_idf

In [109]:
tfidf_transformer = TfidfTransformer()
train_tfidf_ = tfidf_transformer.fit_transform(X_train_)

In [110]:
#class_weight=None
DT_grid_search(hyper_comb,train_tfidf_,y,'DT_grid_search_cv_tfidf_none.csv',None)

100%|██████████| 2240/2240 [36:25<00:00,  1.03it/s]  


File created grid_search_cv


In [111]:
max(list(pd.read_csv('DT_grid_search_cv_tfidf_none.csv')['val_f1']))

0.7659265765489822

In [112]:
#class_weight='balanced'
DT_grid_search(hyper_comb,train_tfidf_,y,'DT_grid_search_cv_tfidf_balanced.csv','balanced')

100%|██████████| 2240/2240 [54:33<00:00,  1.46s/it]  


File created grid_search_cv


In [113]:
max(list(pd.read_csv('DT_grid_search_cv_tfidf_balanced.csv')['val_f1']))

0.7596208120255081